## Import stuff

In [1]:
import numpy as np
from mysticetus.model_driven_reco import *

## NonNegativityTest

Returns `True` if all values in keyword argument `x_new` are not negativ. Otherwise it returns `False`.

In [2]:
nnt = NonNegativityTest()
p1 = np.random.uniform(-0.5, 1.5, [3,])
print(p1)
print(nnt(x_new=p1))

[ 0.39025366 -0.41808109  0.9457882 ]
False


## ConsistencyTest

Returns `True` if holds:

1. The first value (sigma) in keyword argument `x_new` is smaller than the second value (pitch) and
1. The third value (fsize) in keyword argument `x_new` is smaller than pitch.

Otherwise it returns `False`.

In [3]:
ct = ConsistencyTest()
p2 = np.array([0.1, 1., 0.1])
p3 = np.array([1., 1., 0.1])
p4 = np.array([0.1, 1., 1.])
print(ct(x_new=p2))
print(ct(x_new=p3))
print(ct(x_new=p4))

True
False
False


## BoundsTest

Returns `True` if all values in keyword argument `x_new` are in between or the same as their corresponding bounds the test object was initialized with. Otherwise it returns `False`.

In [4]:
pmin = [0., 1., 2.]
pmax = [1., 9., 8.]
bt   = BoundsTest(pmin, pmax)
p5   = np.array([0.5, 9., 3.])
p6   = np.array([1.1, 5., 3.])
p7   = np.array([0.5, 10., 3.])
p8   = np.array([0.5, 5., 9.])
# carefull with lists - they're weird:
p9   =          [0.5, 5., 9.]
print(bt(x_new=p5))
print(bt(x_new=p6))
print(bt(x_new=p7))
print(bt(x_new=p8))
print(bt(x_new=p9))

True
False
False
False
True


## UpperBoundTest

Just like `BoundsTest` but only with an upper boundary.

## HardTest

`HardTest` combines `NonNegativityTest` with `ConsistencyTest`: If keyword argument `x_new` fails any of those to tests, `HardTest` returns `False`. Otherwise it returns `True`.

In [5]:
ht = HardTest()
p10 = np.array([0.5, 1., 0.8])
p11 = np.array([-0.5, 1., 0.8])
p12 = np.array([0.5, 1., 1.1])
print(ht(x_new=p10))
print(ht(x_new=p11))
print(ht(x_new=p12))

True
False
False


## SoftTest

`SoftTest` combines `BoundsTest` with `UpperBoundTest`. Though this might seem useless as the same effect can be achieved with only `BoundsTest` and the right initialization values, it's a good idea still as the `BoundsTest` implements a guess or knowledge of the parameter volume wherin the global minimum should be and, orthogonally, `UpperBoundsTest` implements the size of the real space domain that is taken into account during reconstruction.

In [6]:
domain_size = np.array([5., 5., 5.])
pmin        = np.array([0., 0.1, 0.05])
pmax        = np.array([1., 2.5, 1.25])
st          = SoftTest(domain_size, pmin, pmax)
p13         = np.array([0.1, 0.3, 0.35])
p14         = np.array([1.1, 0.3, 0.35])
print(st(x_new=p13))
print(st(x_new=p14))

True
False


## CombinedTest

Combines `HardTest` and `SoftTest`.

In [7]:
domain_size = np.array([5., 5., 5.])
pmin        = np.array([0., 0.1, 0.05])
pmax        = np.array([1., 2.5, 1.25])
cbt         = CombinedTest(domain_size, pmin, pmax)
p15         = np.array([0.1, 0.3, 0.35])
p16         = np.array([1.1, 0.3, 0.35])
p17         = np.array([0.1, 2.3, 1.2])
print(cbt(x_new=p15))
print(cbt(x_new=p16))
print(cbt(x_new=p17))

False
False
True


## ScaledAxesDisplacement

Like `basinhopping`'s `RandomDisplacement` but with a scaling parameter for each axis.

In [8]:
scales = np.array([1., 10., 100.])
sd     = ScaledAxesDisplacement(scales, stepsize=1.)
p18    = np.array([0., 0., 0.])
print(sd(p18))

[  0.31708692   3.65298578  43.83939149]


## TestedDisplacement

This is a `take_step` functor: It wraps an existing functor of this type and additionally enforces that the returned parametersets pass an `accept_test`-like test.

In [9]:
ht     = HardTest()
scales = np.array([1., 100., 1.])
sd     = ScaledAxesDisplacement(scales, stepsize=1.)
td     = TestedDisplacement(sd, ht)
p19    = np.array([0., 0., 0.])
print(td(p19))

[  3.68462673e+01   3.58797943e+02   4.67416253e-02]
